In [1]:
import numpy as np
import random
import os

# ------------------ 基本井字棋函数 --------------------------
# 创建棋盘
def create_board():
    return np.zeros((3, 3), dtype=int)

# 合法落子
def get_legal_moves(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i, j] == 0]

# 检查谁赢
def check_winner(board):
    for i in range(3):
        if board[i, 0] == board[i, 1] == board[i, 2] != 0:
            return board[i, 0]
        if board[0, i] == board[1, i] == board[2, i] != 0:
            return board[0, i]
    if board[0, 0] == board[1, 1] == board[2, 2] != 0:
        return board[0, 0]
    if board[0, 2] == board[1, 1] == board[2, 0] != 0:
        return board[0, 2]
    return -1 if not np.any(board == 0) else 0

# ------------------ 一些策略 --------------------------
def find_winning_move(board, player):
    """
    寻找能立即取得胜利或需要防守的位置：
    1. 对于每个合法落子点，模拟落子后判断是否能够获胜。
    2. 若存在这样的落子点，则直接返回该点，从而确保AI能快速终结局面或阻止对手获胜。
    """
    for move in get_legal_moves(board):
        temp_board = np.copy(board)
        temp_board[move] = player
        if check_winner(temp_board) == player:
            return move
    return None

def count_potential_lines(board, player):
    """
    统计每条可能获胜路线的潜力得分：
    1. 遍历所有可能的3个格子组成的直线（行、列、对角线）。
    2. 若该直线上己方棋子有2个且空位1个，得分较高（10分），表示距离胜利只差一步。
    3. 若己方棋子有1个且空位2个，得分较低（2分），表示有一定的获胜潜力。
    该策略体现了通过寻找“关键点”来提高胜率的思路。
    """
    lines = [
        [(0,0), (0,1), (0,2)], [(1,0), (1,1), (1,2)], [(2,0), (2,1), (2,2)],
        [(0,0), (1,0), (2,0)], [(0,1), (1,1), (2,1)], [(0,2), (1,2), (2,2)],
        [(0,0), (1,1), (2,2)], [(0,2), (1,1), (2,0)]
    ]
    score = 0
    for line in lines:
        values = [board[i][j] for (i,j) in line]
        player_count = values.count(player)
        empty_count = values.count(0)
        if player_count == 2 and empty_count == 1:
            score += 10  # 差一步胜利
        elif player_count == 1 and empty_count == 2:
            score += 2   # 潜在路线
    return score

def evaluate(board):
    """
    评估当前棋盘局势的好坏：
    - 若AI（编号2）获胜，返回正100分；若对手（编号1）获胜，返回负100分。
    - 若游戏未结束，则返回己方潜力得分减去对手潜力得分，
      反映双方距离胜利的距离，从而指导Alpha-Beta搜索选择最佳落子。
    """
    winner = check_winner(board)
    if winner == 2: return 100
    if winner == 1: return -100
    return count_potential_lines(board, 2) - count_potential_lines(board, 1)

def get_ordered_moves(board):
    """
    返回按照预设优先级排序后的合法落子点：
    - 中心位置（1,1）拥有最大的控制力，所以优先级最高。
    - 接着是四个角落（0,0）、(0,2)、(2,0)、(2,2），占据角落可以限制对手并为未来进攻铺路。
    - 最后是边缘位置。
    通过优化移动顺序，使得Alpha-Beta搜索能更快剪枝，间接提高胜率。
    """
    priority_order = [(1,1), (0,0), (0,2), (2,0), (2,2), (0,1), (1,0), (1,2), (2,1)]
    legal_moves = get_legal_moves(board)
    return sorted(legal_moves, key=lambda x: priority_order.index(x) if x in priority_order else len(priority_order))

# ------------------ Alpha-Beta ----------------
def alphabeta(board, depth, alpha, beta, is_maximizing):
    score = evaluate(board)
    if abs(score) >= 100 or depth >= 9:
        return score
    
    if is_maximizing:
        max_eval = -np.inf
        for move in get_ordered_moves(board):  # 使用优化后的移动顺序
            board[move] = 2
            eval = alphabeta(board, depth+1, alpha, beta, False)
            board[move] = 0
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = np.inf
        for move in get_ordered_moves(board):
            board[move] = 1
            eval = alphabeta(board, depth+1, alpha, beta, True)
            board[move] = 0
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def ai_move(board):
    # 1. 检查立即胜利的机会
    win_move = find_winning_move(board, 2)
    if win_move: return win_move
    
    # 2. 检查需要防守的位置
    block_move = find_winning_move(board, 1)
    if block_move: return block_move
    
    # 3. Alpha-Beta搜索
    best_score = -np.inf
    best_move = None
    alpha, beta = -np.inf, np.inf
    
    for move in get_ordered_moves(board):
        board[move] = 2
        score = alphabeta(board, 0, alpha, beta, False)
        board[move] = 0
        
        if score > best_score:
            best_score = score
            best_move = move
        alpha = max(alpha, score)
    
    return best_move

# ------------------ pk对战 --------------------
def random_move(board):
    moves = get_legal_moves(board)
    return random.choice(moves) if moves else None

def run_test(num_games=100, ai_first=True):
    """运行100局"""
    results = {'win':0, 'lose':0, 'draw':0}
    
    for _ in range(num_games):
        board = create_board()
        ai_turn = ai_first  # 设置先手
        
        while True:
            # AI回合
            if ai_turn:
                move = ai_move(board)
                if move is None: break
                board[move] = 2
            # 随机玩家回合
            else:
                move = random_move(board)
                if move is None: break
                board[move] = 1
            
            # 检查游戏状态
            result = check_winner(board)
            if result != 0:
                break
            
            ai_turn = not ai_turn
        
        # 记录结果
        final_result = check_winner(board)
        if final_result == 2:
            results['win'] += 1
        elif final_result == 1:
            results['lose'] += 1
        else:
            results['draw'] += 1
    
    return results

def print_results(results, title):
    """输出结果"""
    total = sum(results.values())
    print(f"\n===== {title} =====")
    print(f"对局总数: {total}")
    print(f"AI胜率: {results['win']/total:.1%}")
    print(f"AI负率: {results['lose']/total:.1%}") 
    print(f"平局率: {results['draw']/total:.1%}")


if __name__ == '__main__':
    # 先手测试
    first_hand = run_test(num_games=100, ai_first=True)
    print_results(first_hand, "AI先手测试结果")
    
    # 后手测试
    second_hand = run_test(num_games=100, ai_first=False)
    print_results(second_hand, "AI后手测试结果")


===== AI先手测试结果 =====
对局总数: 100
AI胜率: 94.0%
AI负率: 0.0%
平局率: 6.0%

===== AI后手测试结果 =====
对局总数: 100
AI胜率: 80.0%
AI负率: 4.0%
平局率: 16.0%
